<a href="https://colab.research.google.com/github/SarahPendhari/Distilbart-text-summarization-with-gradio/blob/main/Using_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio --quiet

#Import dependencies

In [3]:
import gradio as gr
import nltk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model


In [4]:
checkpoint = "sshleifer/distilbart-cnn-12-6"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


#Define  the function to perform summarization

In [5]:
def summarize_text(text):
    # Split text into sentences
    sentences = nltk.tokenize.sent_tokenize(text)

    # Create chunks
    chunks = []
    chunk = ""
    length = 0
    for sentence in sentences:
        combined_length = len(tokenizer.tokenize(sentence)) + length
        if combined_length <= tokenizer.max_len_single_sentence:
            chunk += sentence + " "
            length = combined_length
        else:
            chunks.append(chunk.strip())
            chunk = sentence + " "
            length = len(tokenizer.tokenize(sentence))
    chunks.append(chunk.strip())

    # Generate summaries for each chunk
    summaries = []
    for chunk in chunks:
        input_ids = tokenizer.encode(chunk, return_tensors="pt", max_length=1024, truncation=True)
        summary_ids = model.generate(input_ids)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    # Combine summaries into a single string
    summarized_text = " ".join(summaries)
    return summarized_text

# Create Gradio interface

In [6]:
iface = gr.Interface(fn=summarize_text,
                      inputs=gr.Textbox(lines=20, label="Input Text"),
                      outputs="text",
                      title="Text Summarization",
                      description="Enter the text you want to summarize and click 'Summarize'.")
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://716f31faa77ec3517e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
